In [1]:
### This workbook is used to scrape the data from the College Hockey News and explore the data## 

## Notes - the site's robots.txt file sets these limits for crawlers
# Crawl-delay: 10 (seconds)
# Request-rate: 1/5 (5 requests every minute)

# Dependencies
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
import json

# Set the URL to scrape
url = 'https://www.collegehockeynews.com/schedules/?season=20202021' # Link to the 2022-2023 season with all results


In [2]:
## explore page structure

# Get the page with requests
# response = requests.get(url)

# # try reading with pandas # Returns odd table structure - going to try BeautifulSoup
# tables = pd.read_html(url)

# tables[0]

# # output as csv
# tables[0].to_csv('../TEMP/2022-2023_season.csv')

In [3]:
## explore page structure

# Get the page with requests
response = requests.get(url)

# Create a BeautifulSoup object
soup = BeautifulSoup(response.text, 'html.parser')

# select the table or tables
tables = soup.find_all('table')

# tables[0] # This appears to be the game results table

In [4]:
# Initialize variables
current_date = None
current_conference = None
game_notes = None

# Initialize an empty list to hold the data
data = []

# Parse the table with BeautifulSoup

rows = soup.find_all('tr')

# Loop through each row to find relevant information
for row in rows:
    # Check for date row
    if row.get('class') == ['stats-section']:
        current_date = row.find('td').text.strip()
    # Check for conference row
    elif row.get('class') == ['sked-header']:
        current_conference = row.find('td').text.strip()
    # Check for game notes
    elif len(row.find_all('td')) == 2:
        game_notes = row.find_all('td')[1].text.strip()
    # Process rows with game data
    elif row.get('valign') == 'top':
        cells = row.find_all('td')
        if len(cells) >= 9:
            home_team = cells[0].text.strip()
            home_team_link = cells[0].find('a')['href'] if cells[0].find('a') else None
            home_score = cells[1].text.strip()
            away_team = cells[3].text.strip()
            away_team_link = cells[3].find('a')['href'] if cells[3].find('a') else None
            away_score = cells[4].text.strip()
            ot = cells[5].text.strip()
            box_link = cells[7].find('a')['href'] if cells[7].find('a') else None
            metrics_link = cells[8].find('a')['href'] if cells[8].find('a') else None
             # Capture Game Notes
            game_notes_cell = cells[-1].find('small')
            game_notes = game_notes_cell.text.strip() if game_notes_cell else None

            # Append data to the list
            data.append([current_date, current_conference, game_notes, home_team, home_team_link, home_score, away_team, away_team_link, away_score, ot, box_link, metrics_link])
            game_notes = None  # Reset game notes for the next row
            

# Create a DataFrame
columns = ['Date', 'Conference', 'Game_Notes', 'Home_Team', 'Home_Team_Link', 'Home_Score', 'Away_Team', 'Away_Team_Link', 'Away_Score', 'OT', 'Box_Link', 'Metrics_Link']
df = pd.DataFrame(data, columns=columns)


In [5]:
## Extract the day of the week from the date and save in new column
df['Day'] = pd.to_datetime(df['Date']).dt.day_name()
# remove day of the week from date
# format data column as YYYY-MM-DD
df['Date'] = pd.to_datetime(df['Date']).dt.strftime('%Y-%m-%d')

### Create a new column for the game ID
## Game ID will be a combination of the date and abbreviated team names

# Function to abbreviate the team names
for row in df.itertuples():
    home_team = row.Home_Team
    away_team = row.Away_Team
    home_team_abbr = home_team.split(' ')[-1]
    away_team_abbr = away_team.split(' ')[-1]
    game_id = f'{row.Date}_{home_team_abbr}_{away_team_abbr}'
    df.loc[row.Index, 'Game_ID'] = game_id

# Create a new column for the game ID
df['Game_ID'] = df['Game_ID'].str.replace(',', '')

# Apply the function to the DataFrame
df['Game_ID'] = df.apply(lambda row: f'{row.Date}_{row.Home_Team}_{row.Away_Team}', axis=1)


C:\Users\Justin\AppData\Local\Temp\ipykernel_11920\2220396521.py:17: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '2020-11-13_Wisconsin_Dame' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.loc[row.Index, 'Game_ID'] = game_id


In [6]:
df.head(20)

# output csv in temp folder
df.to_csv('../TEMP/2020-2021_season.csv')

len(df)

## EMPTY DATAFRAME
# df = pd.DataFrame()


601

In [8]:
# df.head(20)

,Date,Conference,Game_Notes,Home_Team,Home_Team_Link,Home_Score,Away_Team,Away_Team_Link,Away_Score,OT,Box_Link,Metrics_Link,Day,Game_ID
0,2020-11-13,Big Ten,,Wisconsin,/reports/team/Wisconsin/58,2,Notre Dame,/reports/team/Notre-Dame/43,0,,/box/final/20201113/wis/ndm/,/box/metrics.php?gd=84969,Friday,2020-11-13_Wisconsin_Notre Dame
1,2020-11-13,Non-Conference v. D3,,Adrian,None,2,Bowling Green,/reports/team/Bowling-Green/11,6,,/box/final/20201113/adr/bgs/,None,Friday,2020-11-13_Adrian_Bowling Green
2,2020-11-14,Non-Conference,,Arizona State,/reports/team/Arizona-State/61,1,Michigan,/reports/team/Michigan/31,8,,/box/final/20201114/asu/mic/,/box/metrics.php?gd=84971,Saturday,2020-11-14_Arizona State_Michigan
3,2020-11-14,Big Ten,,Wisconsin,/reports/team/Wisconsin/58,5,Notre Dame,/reports/team/Notre-Dame/43,3,,/box/final/20201114/wis/ndm/,/box/metrics.php?gd=84970,Saturday,2020-11-14_Wisconsin_Notre Dame
4,2020-11-15,Non-Conference,,Arizona State,/reports/team/Arizona-State/61,0,Michigan,/reports/team/Michigan/31,3,,/box/final/20201115/asu/mic/,/box/metrics.php?gd=84972,Sunday,2020-11-15_Arizona State_Michigan
5,2020-11-19,Non-Conference,,Long Island,/reports/team/Long-Island/62,3,Holy Cross,/reports/team/Holy-Cross/23,2,ot,/box/final/20201119/liu/hcr/,/box/metrics.php?gd=84819,Thursday,2020-11-19_Long Island_Holy Cross
6,2020-11-19,Non-Conference,,Arizona State,/reports/team/Arizona-State/61,1,Michigan State,/reports/team/Michigan-State/32,1,ot,/box/final/20201119/asu/msu/,/box/metrics.php?gd=84974,Thursday,2020-11-19_Arizona State_Michigan State
7,2020-11-19,Big Ten,,Michigan,/reports/team/Michigan/31,5,Wisconsin,/reports/team/Wisconsin/58,2,,/box/final/20201119/mic/wis/,/box/metrics.php?gd=84975,Thursday,2020-11-19_Michigan_Wisconsin
8,2020-11-19,Big Ten,,Penn State,/reports/team/Penn-State/60,1,Minnesota,/reports/team/Minnesota/34,4,,/box/final/20201119/psu/min/,/box/metrics.php?gd=84973,Thursday,2020-11-19_Penn State_Minnesota
9,2020-11-20,Big Ten,,Penn State,/reports/team/Penn-State/60,2,Minnesota,/reports/team/Minnesota/34,3,,/box/final/20201120/psu/min/,/box/metrics.php?gd=84978,Friday,2020-11-20_Penn State_Minnesota
